In [ ]:
import os
import gzip
import shutil

In [ ]:
os.listdir('//dsw140/CGB-Mapp-Data/newsletter')

### unzip a file from network path and copy to local path

In [ ]:
path = '//dsw140/CGB-Mapp-Data/newsletter/'
for filename in os.listdir(path):
    print(path + filename)
    with gzip.open(path + filename) as f_in:
        with open('.\\data\\file_02.csv', 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    break